<li> primeiro faremos um "Hello World" para ver se esta funcionando </li> 

### Importamos os modulos necessarios para criar o programa

In [1]:
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, IBMQ, Aer

### Configuramos a chamada a API do IBM Q com o token

In [2]:
IBMQ.save_account('ad8ca8c5ae944c27e25cdeb2fc5d9214382f4f5df40f4838a062868c09eb6e1612240d8c9529c9771512154d83eae4d9ab8d2b169dcb253569b770e0b71492e0')

/home/yan/.local/lib/python3.5/site-packages/qiskit/providers/ibmq/credentials/configrc.py:130: UserWarning: Credentials already present. Set overwrite=True to overwrite.
  warnings.warn('Credentials already present. '


Em seguida, criamos um <b>registrador quântico chamado qreg </b> e um <b>registrador clássico chamado creg </b>, com 1 bit cada, e os reunimos em um <b>circuito quântico chamado circ</b>

In [3]:
qreg = QuantumRegister(1)
creg = ClassicalRegister(1)
circ = QuantumCircuit(qreg, creg)

Depois disso, nós vamos aplicar um <b> GATE X no qubit que esta no registrador qreg </b>, medir seu valor e armazená-lo no bit clássico do registrador creg

In [4]:
circ.x(qreg[0])
circ.measure(qreg[0], creg[0])

Então nós configuramos a API para executar esse circuito, o parâmetro <b> shots </b> define quantas vezes a instrução será executada

In [5]:
simulator = Aer.get_backend('qasm_simulator')
# Execute and get counts
result = execute(circ, simulator, shots=2).result()
counts = result.get_counts()

In [6]:
print(counts)

{'1': 2}


Isso significa que, nas duas vezes em que o programa foi executado, o valor do bit no registro clássico foi 1

 <center> <h1> Portões Lógicos</h1> </center>

<li> O portão quântico X atua como um NOT clássico , lançando um qubit no estado |0> para o estado |1> e vice-versa.</li>
<br>
<li> A porta CX atua em dois qubits ao mesmo tempo, sendo um chamado qubit de controle e o outro chamado qubit alvo. Essa porta aplica uma porta X no qubit de alvo apenas se o qubit de controle estiver no estado |1>.</li>
<br>
<li> O portão CCX atua em três qubits por vez, com dois qubits de controle e um qubit alvo. Aplica uma porta X no qubit alvo apenas se ambos os qubits de controle estiverem no estado |1>. </li> 

<h3> Para o gate de soma, precisamos de um gate que aceite três qubits de entrada: o input carry qubit e um qubit de cada um dos addends. O portão executaria uma operação que atingisse os mesmos resultados da tabela abaixo: </h3>

<img src="https://miro.medium.com/max/404/1*6ubvM3j-yop-veQPM99SqQ.png">

A saída é o qubit do registrador b, pois dessa forma, podemos economizar espaço e operações substituindo o registrador b pela soma, em vez de ter outro registrador para armazená-lo. Observando a tabela verdade, podemos dividir a operação em etapas menores: 

<font color='blue'>
<b>
<li> Vire o qubit do registrador b se a entrada de transporte qubit estiver no estado |1>. </li>
<li> Inverta-o novamente se o qubit de A estiver no estado |1>. </li>
</b>
</font>
<br>
<h3> Podemos conseguir esse comportamento usando portas CX :</h3>

<img src="https://miro.medium.com/max/408/1*fythFnrdkXLUnV4xnUTLNQ.png">

<h4>A razão pela qual precisamos de números com menos de 8 dígitos é porque o simulador IBM QISkit possui apenas 32 (qu) bits para usarmos. Se deixarmos que cada número que introduzimos tenha n bits, precisaríamos de:</h4>
<br>
<li>2 n bits para armazenar os dois números</li>
<br>
<li> n transportar bits (veja como os computadores clássicos são adicionados novamente se isso não fizer sentido)</li>
<br>
<li> n + 1 bits clássicos para armazenar o resultado da adição. </li>
<br>
Isso soma 4 n + 1 bits, dos quais 3 n são qubits. Como temos apenas 32 (qu) bits para trabalhar, n pode ser, no máximo, 7,75, o que dá 7 após o arredondamento para o número inteiro mais próximo.
<br><br>
Não precisamos de outro registro para armazenar o resultado (quântico) porque substituímos o segundo número pela soma dos dois números para salvar qubits. Agora verificamos qual número é mais longo e usamos o comprimento do número mais longo para definir os tamanhos de nossos registros, e depois combinamos todos os registros e criamos um circuito quântico.

In [7]:
# first = input('digite o primeiro numero binario com menos de 8 bits: ')
# second = input('digite o segundo numero binario com menos de 8 bits: ')

In [8]:
first = '101110'
second = '100001'

# first = '100111'
# second = '111000'


#verifica qual o maior dos numeros 

lenght_first = len(first)
lenght_second = len(second)

if lenght_first > lenght_second:
    n = lenght_first
else:
    n = lenght_second

In [9]:
#inicializando dois registradores quanticos com n bits cada um (tamanho sera do o do maior numero)
#outro registrador quantico com n+1 que contera o resultado da soma

a = QuantumRegister(n) #primeiro numero
b = QuantumRegister(n+1) #segundo numero, em seguida, soma
c = QuantumRegister(n) #carry bits 

#mais um registrador classico para termos o resultado
cl = ClassicalRegister(n+1) #output classico

#criamos o circuito combinando todos eles 
qc = QuantumCircuit(a,b,c,cl)

Quando iniciamos um circuito quantico todos os qubits estão no estado |0> então precisamos passar para o nosso circuito os bits que digitamos para fazer a soma, e faremos isso aplicando um portao X para alterar os qubits de estado |0> para |1>

In [10]:
#configurando os registradores usando os valores binarios inseridos 
#os circuitos sao configurados fisicamente ao contrario por isso iniciamos um contador para que 1101 possa ser configurado fisicamente como 1011, repare que é o inverso
counter = 0
for i in first:
    if i == "1":
        qc.x(a[lenght_first - (counter+1)])
    counter += 1
counter = 0
for i in second:
    if i == "1":
        qc.x(b[lenght_second - (counter+1)])
    counter += 1

In [11]:
#agora vamos para a adição, primeiro criamos o processo de transporte dos qubits que excedentes da soma para o terceiro circuito (c)
for i in range(n-1):
    qc.ccx(a[i], b[i], c[i+1])
    qc.cx(a[i], b[i])
    qc.ccx(c[i], b[i], c[i+1])

In [12]:
# print(qc)

In [13]:
#For the last iteration of the carry gate, instead of feeding the
#result to c[n], we use b[n], which is why c has only n bits,
#with c[n-1] being the last carry bit
qc.ccx(a[n-1], b[n-1], b[n])
qc.cx(a[n-1], b[n-1])
qc.ccx(c[n-1], b[n-1], b[n])
#Reversing the gate operation performed on b[n-1]
qc.cx(c[n-1], b[n-1])
#Reversing the gate operations performed during the carry gate
#implementations, which is done to reset all carry bits to 
#the |0> state
for i in range(n-1):
    qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    qc.cx(a[(n-2)-i], b[(n-2)-i])
    qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    #These two operations act as a sum gate; if a control bit is 
    #in the |1> state then the target bit b[(n-2)-i] is flipped
    qc.cx(c[(n-2)-i], b[(n-2)-i])
    qc.cx(a[(n-2)-i], b[(n-2)-i])

In [14]:
#Measure qubits
for i in range(n+1):
    qc.measure(b[i], cl[i])

In [15]:
#Set chosen backend and execute job
num_shots = 2 #Setting the number of times to repeat measurement
job = execute(qc, backend=Aer.get_backend('qasm_simulator'), shots=num_shots)
#Get results of program
job_stats = job.result().get_counts()
print(job_stats)   

{'1001111': 2}


In [16]:
print(qc)

                                                                               »
q1_0: |0>────────────■─────────────────────────────■───────────────────────────»
         ┌───┐       │                             │                           »
q1_1: |0>┤ X ├───────┼────■────────────────────────┼────■──────────────────────»
         ├───┤       │    │                        │    │                      »
q1_2: |0>┤ X ├───────┼────┼────■───────────────────┼────┼────■─────────────────»
         ├───┤       │    │    │                   │    │    │                 »
q1_3: |0>┤ X ├───────┼────┼────┼────■──────────────┼────┼────┼────■────────────»
         └───┘       │    │    │    │              │    │    │    │            »
q1_4: |0>───────■────┼────┼────┼────┼────■─────────┼────┼────┼────┼────────────»
         ┌───┐  │    │    │    │    │    │         │    │    │    │            »
q1_5: |0>┤ X ├──┼────┼────┼────┼────┼────┼────■────┼────┼────┼────┼────■───────»
         ├───┤  │    │    │ 